<a href="https://colab.research.google.com/github/NLPetroni/assignment_two/blob/main/solution_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and downloads



In [3]:
import numpy as np
import pandas as pd
import sys
import os

%cd /content
!rm -rf assignment_two &> /dev/null
!git clone https://github.com/NLPetroni/assignment_two &> /dev/null
%cd assignment_two
sys.path.append(os.getcwd())
!git clone https://gitlab.com/sasso-effe/nlp-assignment-data.git &> /dev/null
!mv nlp-assignment-data/embedding_matrix.npy res/embedding_matrix.npy
!rm -rf nlp-assignment-data


/content
/content/assignment_two


In [4]:
from src import utils
import re
from functools import reduce
import nltk
from nltk.corpus import stopwords
from typing import List, Callable, Dict
import random
import torch
from torch import nn

In [6]:
utils.download_data('dataset')
train_set = pd.read_csv("dataset/train_pairs.csv")
val_set = pd.read_csv("dataset/val_pairs.csv")
test_set = pd.read_csv("dataset/test_pairs.csv")

Download completed!
Extracting dataset...
Extraction completed!


In [7]:
print(train_set.columns)
print("Total rows of the train set: {:d}".format(len(train_set)))
print("Total rows of the validation set: {:d}".format(len(val_set)))
print("Total rows of the test set: {:d}".format(len(test_set)))

Index(['Unnamed: 0', 'Claim', 'Evidence', 'ID', 'Label'], dtype='object')
Total rows of the train set: 121740
Total rows of the validation set: 7165
Total rows of the test set: 7189


In [8]:
train_set['Label'].value_counts()

SUPPORTS    89389
REFUTES     32351
Name: Label, dtype: int64

In [9]:
print(train_set.iloc[0]['Evidence'])

2	Hemsworth has also appeared in the science fiction action film Star Trek -LRB- 2009 -RRB- , the thriller adventure A Perfect Getaway -LRB- 2009 -RRB- , the horror comedy The Cabin in the Woods -LRB- 2012 -RRB- , the dark-fantasy action film Snow White and the Huntsman -LRB- 2012 -RRB- , the war film Red Dawn -LRB- 2012 -RRB- , and the biographical sports drama film Rush -LRB- 2013 -RRB- .	Star Trek	Star Trek (film)	A Perfect Getaway	A Perfect Getaway	The Cabin in the Woods	The Cabin in the Woods	Snow White and the Huntsman	Snow White and the Huntsman	Red Dawn	Red Dawn (2012 film)	Rush	Rush (2013 film)


# Dataset pre-processing and conversion

In [10]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;\t-]')
GOOD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
BAD_SYMBOLS_RE = re.compile('(-LRB-)|(-RRB-)|(-LSB-)|(-RSB-)')
INSIDE_SQAURE_BRACKETS_RE = re.compile('(-LSB-).*?(-RSB-)')

try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

def remove_inside_square_brackets(text: str) -> str:
    return INSIDE_SQAURE_BRACKETS_RE.sub('', text)

def remove_bad_symbols(text: str) -> str:
    return BAD_SYMBOLS_RE.sub('', text)

def remove_final_tags(text: str) -> str:
   return re.sub('\.\t.*?$', '', text) 

def lower(text: str) -> str:
    """
    Transforms given text to lower case.
    Example:
    Input: 'I really like New York city'
    Output: 'i really like new your city'
    """

    return text.lower()

def replace_special_characters(text: str) -> str:
    """
    Replaces special characters, such as paranthesis,
    with spacing character
    """

    return REPLACE_BY_SPACE_RE.sub(' ', text)

def replace_br(text: str) -> str:
    """
    Replaces br characters
    """

    return text.replace('</br>', '')

def filter_out_uncommon_symbols(text: str) -> str:
    """
    Removes any special character that is not in the
    good symbols list (check regular expression)
    """

    return GOOD_SYMBOLS_RE.sub('', text)

def remove_stopwords(text: str) -> str:
    return ' '.join([x for x in text.split() if x and x not in STOPWORDS])


def strip_text(text: str) -> str:
    """
    Removes any left or right spacing (including carriage return) from text.
    Example:
    Input: '  This assignment is cool\n'
    Output: 'This assignment is cool'
    """

    return text.strip()

def split_text(text: str) -> List:
  return text.split()

PREPROCESSING_PIPELINE = [
                          remove_inside_square_brackets,
                          remove_bad_symbols,
                          lower,
                          remove_final_tags,
                          replace_special_characters,
                          filter_out_uncommon_symbols,
                          remove_stopwords,
                          strip_text,
                          split_text
                          ]

# Anchor method

def text_prepare(text: str,
                 filter_methods: List[Callable[[str], str]] = None) -> str:
    """
    Applies a list of pre-processing functions in sequence (reduce).
    Note that the order is important here!
    """

    filter_methods = filter_methods if filter_methods is not None else PREPROCESSING_PIPELINE

    return reduce(lambda txt, f: f(txt), filter_methods, text)


# In the evidences there is an id at the beginning of the sequence which is
# removed with the splice [:1]
train_set['Evidence'] = train_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
train_set['Claim'] = train_set['Claim'].apply(lambda txt: text_prepare(txt))

val_set['Evidence'] = val_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
val_set['Claim'] = val_set['Claim'].apply(lambda txt: text_prepare(txt))

test_set['Evidence'] = test_set['Evidence'].apply(lambda txt: text_prepare(txt)[1:])
test_set['Claim'] = test_set['Claim'].apply(lambda txt: text_prepare(txt))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
voc_evidence = [item for sublist in train_set[:]['Evidence'] for item in sublist]
voc_claim = [item for sublist in train_set[:]['Claim'] for item in sublist]
vocabulary = list(set(voc_evidence + voc_claim))

def tokenize(input: List) -> torch.Tensor:
  result = list(map(lambda x: vocabulary.index(x), input))
  return torch.tensor(result)

def detokenize(input: torch.Tensor) -> List:
  result = input.tolist()
  result = list(map(lambda x: vocabulary[x], result))
  return result

## Glove

In [12]:
import pickle

if (os.path.exists("res/vocabulary.pkl") and os.path.exists("res/embedding_matrix.npy")):
  print('The vocabulary and the embedding matrix are already present. Loading them...')
  with open('res/vocabulary.pkl', 'rb') as f:
    VOCABULARY = pickle.load(f)
  EMBEDDING_MATRIX = np.load("res/embedding_matrix.npy")
  print("Done!")
  
else:
  print("The vocabulary and the embedding matrix are NOT present. Creating them...")
  TRAIN_VOC = set(vocabulary)
  voc_evidence = [item for sublist in val_set[:]['Evidence'] for item in sublist]
  voc_claim = [item for sublist in val_set[:]['Claim'] for item in sublist]
  VAL_VOC = set(voc_evidence + voc_claim)

  inputs = train_set[:]['Evidence'].tolist() + train_set[:]['Claim'].tolist()
  glove_voc, embedding_matrix = utils.get_glove(number_token=False)
  vocabulary, embedding_matrix = utils.add_oov(glove_voc, TRAIN_VOC, embedding_matrix, inputs)
  inputs = val_set[:]['Evidence'].tolist() + val_set[:]['Claim'].tolist()
  vocabulary, embedding_matrix = utils.add_oov(vocabulary, VAL_VOC, embedding_matrix, inputs)

  with open("vocabulary.pkl", "wb") as file:
    pickle.dump(vocabulary, file)
  np.save("embedding_matrix.npy", embedding_matrix)
  print("Vocabulary and embedding matrix created! Remember to download the generated files.")

The vocabulary and the embedding matrix are already present. Loading them...
Done!


# Model definition

In [51]:
def get_classifier(name:str,
                    w_in: int,
                    w_hidden: int,
                    w_out: int) -> nn.Sequential:
    """Gets a sequential container with a linear+relu+linear classifier

    Args:
        name: the name prefix to append to each layer in the container.
        w_in: the number of the input features.
        w_hidden: the number of internal weights
        w_out: the number of the output features.

    Returns: the created sequential.
    """
    container = nn.Sequential()
    container.add_module(f'{name}_fc1', nn.Linear(in_features=w_in, out_features=w_hidden))    
    container.add_module(f'{name}_ReLU', nn.ReLU(inplace=True))
    container.add_module(f'{name}_fc2', nn.Linear(in_features=w_hidden, out_features=w_out))    
    return container

 
class RNNEncoder(torch.nn.Module):

  def __init__(self, input_size, hidden_size, num_layers, rnn_type='elman', output_state='last'):
    super().__init__()
    types = {'elman': nn.RNN, 'lstm': nn.LSTM, 'gru': nn.GRU}
    states = {
        'last': lambda x: x[-1],
        'avg': lambda x: torch.mean(x, dim=0)}
    
    try:
      self.output_state_fn = states[output_state]
    except:
      valid_states = list(states.keys())
      raise ValueError(f"wrong type '{output_state}', must be in {valid_states}")

    try:
      rec_module = types[rnn_type]
    except:
      valid_types = list(types.keys())
      raise ValueError(f"wrong type '{rnn_type}', must be in {valid_types}")
    self.rec_module = rec_module(input_size=input_size, hidden_size=hidden_size,
                                 bidirectional=True, batch_first=False,
                                 num_layers=num_layers)
    
  def forward(self, x):
    output, _ = self.rec_module(x)
    return self.output_state_fn(output)

class BagOfVectorsEncoder(torch.nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    # TODO: check if the mean is computed on the right axis
    return torch.mean(x, axis=1)


In [52]:
class FactChecker(torch.nn.Module):

  def __init__(self, embedding_matrix, encoder, merger, rnn_type=None, rnn_output=None, rec_size=1, hid_size=50):
    """
      A recurrent network performing Neural Language Inference (Fact Checking).
      Params:
        embedding_matrix: the embedding matrix for word embedding
        encoder: [rnn, mlp, bag], the encoder to compute the sentence embedding
        merger: [concatenation, sum, mean], the multi-input merging strategy
        RNNEncoder params, only relevant if encoder==rnn:
          rnn_type: [elman, lstm, gru], the RNN architecure used in the encoder
          rnn_output: [last, avg], the function to compute the sentence encoding from the RNN hidden states
          rec_size: int, the number of layers in the rnn
          hid_size: int, the hidden size of the rnn

    """
    super().__init__()
    self.hid_size = hid_size

    # Word embedding
    emb_size = embedding_matrix.shape[1]
    self.emb_layer = nn.Embedding.from_pretrained(torch.as_tensor(embedding_matrix))

    # Sentence embedding
    if encoder == 'rnn':
      self.encoder = RNNEncoder(emb_size, hid_size, rec_size, rnn_type=rnn_type, output_state=rnn_output)
    elif encoder == 'mlp':
      pass #TODO: implement
    elif encoder == 'bag':
      self.encoder = BagOfVectorsEncoder()
    else:
      raise ValueError(f"Wrong encoder '{encoder}', must be in ['rnn', 'mlp', 'bag']")

    # Merging
    merging_strategies = {
        # TODO: check that these function operate on the right axis
        'concatenation': torch.cat,
        'sum': torch.add,
        'mean': torch.mean
    }
    try:
      merging_fn = merging_strategies[merger]
    except:
      valid_strategies = list(merging_strategies.keys())
      raise ValueError(f"wrong type '{merger}', must be in {valid_strategies}")
    self.merger = merging_fn

    # Classifier
    self.classifier = get_classifier('classifier', w_in=2*self.hid_size, w_hidden=self.hid_size, w_out=2)

  def __call__(self, x):
    vecs = self.emb_layer(x).float()
    output = self.encoder(vecs)
    # it works until here (probably)
    return output


    

In [53]:
model = FactChecker(EMBEDDING_MATRIX, 'rnn', 'sum', rnn_type='elman', rnn_output='last', rec_size=1, hid_size=50)

In [55]:
x = torch.randint(2000, (10,10))
print(model(x).shape)


torch.Size([10, 100])


# Training

In [ ]:
def training_step(model, optimizer, loss_fn, data_loader, device):
  model.train()

  for (input, target) in data_loader:
    #forward
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = loss_fn(output, target)
    loss_value = loss.item()

    if not math.isfinite(loss_value):
      print(f"Loss is {loss_value}, stopping training")
      exit(1)

    #backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()    


In [ ]:
def train(optimizer_name, lr, loss_fn, data_loader, device, n_epochs, verbose, batch_size):
  '''wandb.login(key=utils.get_wandbkey()) # TODO: implement getter
  run = wandb.init(project="assignment-two", entity="nlpetroni", reinit=True, config=cfg_dict)
  wandb.define_metric("train_step")
  wandb.define_metric("epoch")
  wandb.define_metric('train/loss', step_metric="train_step", summary="min")
  wandb.define_metric("valid/loss", step_metric="epoch", summary="min")
  wandb.define_metric("valid/accuracy", step_metric="epoch", summary="max")'''

  train_dl = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
  valid_dl = torch.utils.data.DataLoader(val_set, batch_size=batch_size)

  model = FactChecker(EMBEDDING_MATRIX, 'rnn', rnn_type='lstm', rnn_output='last')
  wandb.watch(model, log_graph=True)
  if verbose:
    print(summary(model))

  if optimizer_name == 'rmsprop':
    optimizer = torch.optim.RMSprop(params, lr=cfg.LR, alpha=0.99, momentum=0.5, weight_decay=0)
  elif optimizer_name == 'adam':
    optimizer = torch.optim.Adam(params, lr=cfg.LR, betas=(0.9, 0.999), weight_decay=0)
  else:
    raise ValueError(f'wrong optim {optimizer_name}, either rmsprop or adam')

  loss = nn.NLLLoss()
  train_step = 0
  print('STARTING TRAINING')

  for epoch in range(n_epochs):
    training_step(model, optimizer, loss, train_dl, device)

    

In [ ]:
v, m = train(0, 0, 0, 0, 0, 0)
print(list(m.keys())[0])

In [ ]:
print(m[-1])

In [ ]:
inputs = train_set[:]['Evidence'].tolist() + train_set[:]['Claim'].tolist()
print(inputs[0])

In [ ]:
glove_voc = utils.get_glove(number_token=False)

In [ ]:
list(glove_voc[0].keys())[0]